<a href="https://colab.research.google.com/github/castrokelly/Data-Science/blob/main/Abordagem_Ensemble_para_Detec%C3%A7%C3%A3o_e_Classifica%C3%A7%C3%A3o_de_Anomalias_em_Po%C3%A7os_de_Petr%C3%B3leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Ensemble para Detecção e Classificação de Anomalias em Poços de Petróleo: um Estudo Aplicado ao Dataset 3W

**Centro de Ciências Matemáticas Aplicadas à Indústria (CeMEAI)**</br>
**Instituto de Ciências Matemáticas e de Computação (ICMC)**</br>
**Universidade de São Paulo**</br>
</br>
Aluna: **Kelly Christine Alvarenga de Castro**
Área de concentração: Ciências de Dados
Orientador: **Prof. Dr. Cláudio Fabiano Motta Toledo**
</br>
---

Este Notebook apresenta o desenvolvimento de um pipeline de aprendizado de máquina para detectar e classificar anomalias em poços de petróleo utilizando o dataset 3W.

A abordagem utiliza um modelo ensemble que, em primeiro lugar, decide se o estado é anômalo (classificação binária: normal vs. anômalo) e, caso seja anômalo, classifica o tipo de evento (rótulos de 1 a 9). O treinamento é realizado com dados de um poço e a validação das métricas é efetuada com dados de outro poço.



Clonagem do repositório contendo o dataset 3W e instalação das bibliotecas necessárias:

In [14]:
!git clone https://github.com/petrobras/3W.git

fatal: destination path '3W' already exists and is not an empty directory.


In [15]:
!pip install scikeras #para a integração do Keras com o scikit-learn
!pip install PyWavelets #para a transformação wavelet

Importação das bibliotecas necessárias para manipulação dos dados, pré-processamento, construção e avaliação dos modelos:

In [16]:
# Importação das Bibliotecas Necessárias
import os
import glob
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import pywt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Para o modelo LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier


### Funções para Carregamento e Pré-processamento dos Dados

Exibição da Estrutura do DataFrame

In [17]:
# Carregando o arquivo Parquet em um DataFrame usando pandas
example_file_path = "/content/3W/dataset/1/WELL-00001_20140124083303.parquet"
df_example = pd.read_parquet(example_file_path)

# Exibindo as primeiras 5 linhas do DataFrame para visualização da estrutura
print("Estrutura do DataFrame:")
display(df_example.head())


Estrutura do DataFrame:


,ABER-CKGL,ABER-CKP,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,ESTADO-W1,ESTADO-W2,...,PT-P,P-TPT,QBS,QGL,T-JUS-CKP,T-MON-CKP,T-PDG,T-TPT,class,state
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-01-24 08:33:03,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,18433410.0,NaN,0.0,74.80221,NaN,0.0,116.8744,<NA>,<NA>
2014-01-24 08:33:04,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,18433410.0,NaN,0.0,74.80221,NaN,0.0,116.8751,<NA>,<NA>
2014-01-24 08:33:05,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,18433410.0,NaN,0.0,74.80221,NaN,0.0,116.8757,<NA>,<NA>
2014-01-24 08:33:06,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,18433410.0,NaN,0.0,74.80221,NaN,0.0,116.8764,<NA>,<NA>
2014-01-24 08:33:07,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,18433410.0,NaN,0.0,74.80221,NaN,0.0,116.8770,<NA>,<NA>


In [18]:
# Carregar os dados do dataset 3W
def load_data(folder_path):
    files = glob.glob(folder_path + "/*.parquet")
    dataframes = [pq.read_table(file).to_pandas() for file in files]
    return pd.concat(dataframes, ignore_index=True)

In [19]:
dataset_path = "/content/3W/dataset/9"
df = load_data(dataset_path)

print(df.head())

   ABER-CKGL  ABER-CKP  ESTADO-DHSV  ESTADO-M1  ESTADO-M2  ESTADO-PXO  \
0        NaN       NaN          NaN        NaN        NaN         NaN   
1        NaN       NaN          NaN        NaN        NaN         NaN   
2        NaN       NaN          NaN        NaN        NaN         NaN   
3        NaN       NaN          NaN        NaN        NaN         NaN   
4        NaN       NaN          NaN        NaN        NaN         NaN   

   ESTADO-SDV-GL  ESTADO-SDV-P  ESTADO-W1  ESTADO-W2  ...  PT-P         P-TPT  \
0            NaN           NaN        NaN        NaN  ...   NaN  9.448843e+06   
1            NaN           NaN        NaN        NaN  ...   NaN  9.448844e+06   
2            NaN           NaN        NaN        NaN  ...   NaN  9.448842e+06   
3            NaN           NaN        NaN        NaN  ...   NaN  9.448854e+06   
4            NaN           NaN        NaN        NaN  ...   NaN  9.448853e+06   

   QBS       QGL  T-JUS-CKP  T-MON-CKP  T-PDG      T-TPT  class  state  
0